<a href="https://colab.research.google.com/github/Santiago-R/aupa.ai/blob/main/lm-hackers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [A hacker's guide to Language Models](https://colab.research.google.com/github/fastai/lm-hackers/blob/main/lm-hackers.ipynb#scrollTo=0b017bfc-5be0-4e41-9fa1-9f685c3b0de5)


## Initial restart

In [ ]:
import os
import importlib
accelerate_spec = importlib.util.find_spec('accelerate')
bitsandbytes_spec = importlib.util.find_spec('bitsandbytes')

found = accelerate_spec is not None and bitsandbytes_spec is not None

if found: print('Dependencies installed in previous run ✔️')

else:
    !pip install accelerate -qq
    !pip install bitsandbytes -qq
    print('\nDependency installation requires restart --> Please kill runtime 💀')

In [ ]:
if not found:
    os.kill(os.getpid(), 9)

## Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')  # , force_remount=True)

Mounted at /content/drive


In [3]:
# import tokenize
# from io import BytesIO

In [4]:
# from pathlib import Path
# path = Path("/content/drive/MyDrive/LLM")

## The OpenAI API

In [5]:
# Load OpenAI api key as environvent variable (from Drive's api_keys.env)
!pip install python-dotenv -qq
from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/LLM/api_keys.env')

True

In [6]:
!pip install openai -qq
from openai import ChatCompletion, Completion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00


In [7]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies whenever possible."
question = "What is money?"

c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": question}])

- [Model options](https://platform.openai.com/docs/models)

In [8]:
def response(c):
    try:
        return c['choices'][0]['message']['content']
    except KeyError:
        return c['choices'][0]['text']

In [9]:
response(c)

"Hey mate, money is like the lifeblood of our economy. It's a medium of exchange that we use to buy goods and services. It's like the fuel that keeps the economic engine runnin'. Without money, it'd be like tryin' to drive a car without any petrol in the tank – you ain't goin' anywhere! People earn money by workin' and they can use it to pay for stuff they need or want. It's a pretty crucial element in our modern society."

In [10]:
print(c.usage)

{
  "prompt_tokens": 31,
  "completion_tokens": 102,
  "total_tokens": 133
}


In [11]:
0.002 / 1000 * 150  # GPT 3.5

0.0003

In [12]:
0.03 / 1000 * 150  # GPT 4

0.0045

In [13]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"},
              {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
              {"role": "user", "content": "Really? In what way?"}])

In [14]:
response(c)

"Ah, let me break it down for ya. Money is a bit like kangaroos because just like kangaroos bounce around from one place to another, money bounces from one person to another in an economy. It's a medium of exchange that allows us to trade goods and services. Just like kangaroos have different sizes, types, and strengths, money comes in different forms too, such as notes, coins, or even digital currencies. And just as kangaroos can be powerful and influential in a dynamic ecosystem, money has a significant impact on the economy and can determine our lifestyles and opportunities. So, in a nutshell, money's a bit like a kangaroo – it moves around, comes in different shapes and sizes, and has an important role in our lives down under."

In [15]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [16]:
response(askgpt('What is the meaning of life?', system=aussie_sys))

"Well, mate, the meaning of life is a real ripper of a question, isn't it? Now, I'm no philosopher, but I reckon the meaning of life is all about finding your true blue purpose and living it to the fullest. It's about chasing your dreams, forging meaningful relationships, and finding happiness in the little things. Life's like a game of footy, ya know? You gotta give it your all, have a crack, and enjoy the ride. At the end of the day, it's up to each of us to create our own meaning and make our time here on Earth worth it. So, get out there and have a bloody good one, mate!"

- [Limits](https://platform.openai.com/docs/guides/rate-limits/what-are-the-rate-limits-for-our-api)

Created by Bing:

In [17]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return ChatCompletion.create(model=model, messages=msgs)
    except openai.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [18]:
response(call_api("What's the world's funniest joke? Has there ever been any scientific analysis?"))

'Determining the world\'s funniest joke is highly subjective, as humour varies between individuals and cultures. However, there have been attempts to analyze jokes scientifically. \n\nIn 2002, a study conducted by Richard Wiseman, a British psychologist, involved a panel of 40,000 people from 70 countries sharing their favorite jokes. The participants then ranked the jokes based on their funniness. The winning joke from this study was the following:\n\n"Two hunters are out in the woods in Africa when one of them collapses. He doesn\'t seem to be breathing and his eyes are glazed. The other guy whips out his phone and calls the emergency services. He gasps, \'My friend is dead! What can I do?\' The operator says, \'Calm down, I can help. First, let\'s make sure he\'s dead.\' There is silence, then a gunshot is heard. Back on the phone, the guy says, \'Okay, now what?\'"\n\nWhile this joke was ranked as the funniest according to Wiseman\'s study, it\'s important to remember that humor is

In [19]:
c = Completion.create(prompt="Australian Jeremy Howard is ",
                      model="gpt-3.5-turbo-instruct", echo=True, logprobs=5)

In [20]:
response(c)

'Australian Jeremy Howard is 200 paces from walking on water.\n\nThe father of four from Port Macqu'

## Create our own code interpreter

In [21]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

#### Example with sums function

In [22]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [23]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [24]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'title': 'Input for `sums`',
  'type': 'object',
  'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'default': 1, 'type': 'integer'}},
  'required': ['a']}}

In [25]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [26]:
c

<OpenAIObject chat.completion id=chatcmpl-86SPjA4nqjFE76quyX7H6EyLZMrdN at 0x7ff53e421620> JSON: {
  "id": "chatcmpl-86SPjA4nqjFE76quyX7H6EyLZMrdN",
  "object": "chat.completion",
  "created": 1696549883,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "sums",
          "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 22,
    "total_tokens": 105
  }
}

In [27]:
m = c.choices[0].message
m

<OpenAIObject at 0x7ff53e82df30> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

In [28]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


In [29]:
funcs_ok = {'sums', 'python'}

In [30]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [31]:
call_func(c)

9

In [32]:
import ast

def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None

    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [33]:
run("""
a=1
b=2
a+b
""")

3

In [34]:
def python(code:str, safe_mode:bool=False):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    if safe_mode:
        go = input(f'Proceed with execution?\n```\n{code}\n```\n')
        if go.lower()!='y': return '#FAIL#'
    return run(code)

In [35]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [36]:
c

<OpenAIObject chat.completion id=chatcmpl-86SPkXTnLKdlZfcNhvjkt2Ct1YWYK at 0x7ff53e4433d0> JSON: {
  "id": "chatcmpl-86SPkXTnLKdlZfcNhvjkt2Ct1YWYK",
  "object": "chat.completion",
  "created": 1696549884,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "python",
          "arguments": "{\n  \"code\": \"import math\\nmath.factorial(12)\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 21,
    "total_tokens": 103
  }
}

In [37]:
call_func(c)

479001600

#### Using Python's exec & eval

In [38]:
exec_schema = {
    'name': 'exec',
    'description': 'Execute the given source Python code',
    'parameters': {
        'title': 'Input for `exec`',
        'type': 'object',
        'properties': {'source': {'title': 'S', 'type': 'string'}},
        'required': ['source']}}

In [39]:
def code_response(c, repl=True):
    txt_out = response(c)
    if txt_out == None: txt_out = ''
    if 'function_call' not in c['choices'][0]['message'].keys():
        print(txt_out)
        return  # No code output
    code = c['choices'][0]['message']['function_call']['arguments']
    if code[0] == '{': code = json.loads(code)['source']
    txt_out += f'\n==========\n{code}\n==========\n>>> '
    if repl:
        code_body = '\n'.join(code.split('\n')[:-1])
        code_footer = code.split('\n')[-1]
        exec(code_body, locals())
        result = eval(code_footer, locals())
        txt_out += str(result)
    else:
        exec(code, locals())
        result = None
    print(txt_out)
    return result

In [40]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [41]:
factorial_result = code_response(c, repl=True)


import math

math.factorial(12)
>>> 479001600


In [42]:
factorial_result

479001600

###### Using result in later calls

In [43]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[exec_schema],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "exec", "content": str(factorial_result)}])

In [44]:
print(response(c))

The factorial of 12 is 479,001,600.


###### And we didn't break its basic use!

In [45]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [46]:
code_response(c)

The capital of France is Paris.


## PyTorch and Huggingface

In [47]:
!pip install transformers -qq
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00


- [HF leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- [fasteval](https://fasteval.github.io/FastEval/)

In [48]:
mn = "meta-llama/Llama-2-7b-hf"

In [49]:
# !huggingface-cli login
# # Model access needs granted at https://huggingface.co/meta-llama/Llama-2-7b-hf

# TODO: Login in environment variable

In [50]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [52]:
toks

{'input_ids': tensor([[    1,  5677,  6764, 17430,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [53]:
tokr.batch_decode(toks['input_ids'])

['<s> Jeremy Howard is a ']

In [54]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

CPU times: user 10.3 s, sys: 900 ms, total: 11.2 s
Wall time: 21.5 s


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29896, 29899,
          6360, 29899,  1025,  3082,   319,  2801,   515,  4602, 10722, 29892,
          8046, 29892]])

In [55]:
tokr.batch_decode(res)

['<s> Jeremy Howard is a 31-year-old American Actor from Los Angeles, California,']

In [56]:
# # OutOfMemoryError
# model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [57]:
# %%time
# res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
# res

In [58]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

ImportError: ignored

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
mn = 'TheBloke/Llama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [ ]:
gen(prompt, 50)

[StableBeluga-7B](https://huggingface.co/stabilityai/StableBeluga-7B)

In [ ]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [ ]:
ques = "Who is Jeremy Howard?"

In [ ]:
gen(mk_prompt(ques), 150)

[OpenOrca/Platypus 2](https://huggingface.co/Open-Orca/OpenOrca-Platypus2-13B)

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques), 150)

In [ ]:
!pip install wikipedia-api
from wikipediaapi import Wikipedia

In [ ]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [ ]:
print(jh_page[:500])

In [ ]:
len(jh_page.split())

In [ ]:
# def mk_prompt_context(question, context):
#     return f"""Answer the question with the help of the provided context.\n\n## Context\n\n{context}\n\n## Question\n\n{question}"""

In [ ]:
res = gen(mk_prompt(ques_ctx), 300)

In [ ]:
print(res[0].split('### Assistant:\n')[1])

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
jh = jh_page.split('\n\n')[0]
print(jh)

In [ ]:
tb_page = wiki.page('Tony_Blair').text.split('\nReferences\n')[0]

In [ ]:
tb = tb_page.split('\n\n')[0]
print(tb[:380])

In [ ]:
q_emb,jh_emb,tb_emb = emb_model.encode([ques,jh,tb], convert_to_tensor=True)

In [ ]:
tb_emb.shape

In [ ]:
import torch.nn.functional as F

In [ ]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

In [ ]:
F.cosine_similarity(q_emb, tb_emb, dim=0)

### Private GPTs

- [Sooo many](https://github.com/h2oai/h2ogpt/blob/main/docs/README_LangChain.md#what-is-h2ogpts-langchain-integration-like)

## Fine tuning

In [ ]:
import datasets

[knowrohit07/know_sql](https://huggingface.co/datasets/knowrohit07/know_sql)

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

In [ ]:
ds

In [ ]:
trn = ds['train']
trn[3]

`accelerate launch -m axolotl.cli.train sql.yml`

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

In [ ]:
print(sql_prompt(tst))

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
ax_model = '/home/jhoward/git/ext/axolotl/qlora-out'

In [ ]:
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [ ]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',
                                             torch_dtype=torch.bfloat16, device_map=0)
model = PeftModel.from_pretrained(model, ax_model)
model = model.merge_and_unload()
model.save_pretrained('sql-model')

In [ ]:
toks = tokr(sql_prompt(tst), return_tensors="pt")

In [ ]:
res = model.generate(**toks.to("cuda"), max_new_tokens=250).to('cpu')

In [ ]:
print(tokr.batch_decode(res)[0])

## [llama.cpp](https://github.com/abetlen/llama-cpp-python)

[TheBloke/Llama-2-7b-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF)

In [ ]:
!pip install llama_cpp_python -qq
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path="content/llamacpp/llama-2-7b-chat.Q4_K_M.gguf")

In [ ]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)

In [ ]:
print(output['choices'])